In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import numpy.ma as ma
import matplotlib.mlab as mlab
from scipy.stats import norm
from scipy.optimize import curve_fit
from collections import defaultdict

In [2]:
%pylab
%matplotlib qt

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
knots = True

if knots == True:
    scale = 1.94384
    units = 'knots'
    sunits = 'knots'
if knots == False:
    scale = 1.
    units = 'm/s'
    sunits = 'mps'

In [16]:
wind_speed = dict()
temperature = dict()
wind_speedSP = dict()
wind_speedH = dict()
wind_dir = dict()
months = dict()
monthsSP = dict()
monthsH = dict()
day = dict()
daySP = dict()
dayH = dict()
hour = dict()
hourSP = dict()
hourH = dict()
precipitation = dict()


#YEAR = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]
YEAR = [2012, 2013, 2014, 2015, 2016, 2017, 2018]
#2008 starts on a different month, need to make sure plot reflects that

month = ['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
numDays = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
#method 1
#time = dict()
for y in YEAR:
    
    file1 = '../telescope_control/data_aquisition/weather_data/meteorological/%s/met_sum_insitu_1_obop_hour_%d.txt' % (y, y)
    #fileSP = '../meteorology/data/SP/%d/hourly/met_spo_insitu_1_obop_hour_%d.txt' % (y, y)
    #fileH = '../meteorology/data/Hawaii/%d/hourly/met_mlo_insitu_1_obop_hour_%d.txt' % (y, y)
    
    results = []
    with open(file1) as inputfile:
        for line in inputfile:
            results.append(line.strip().split())
    '''
    resultsSP = []
    with open(fileSP) as inputfile:
        for line in inputfile:
            resultsSP.append(line.strip().split())
    resultsH = []
    with open(fileH) as inputfile:
        for line in inputfile:
            resultsH.append(line.strip().split())
    '''
    direction = []
    speed = []
    temp = []
    speedSP = []
    speedH = []
    m = []
    mSP = []
    mH = []
    d = []
    dSP = []
    dH = []
    h = []
    hSP = []
    hH = []
    prec = []
    #times = []

    for i in range(len(results)):
        direction.append(float(results[i][5]))
        speed.append(float(results[i][6]))
        m.append(float(results[i][2]))
        d.append(float(results[i][3]))
        h.append(float(results[i][4]))
        temp.append(float(results[i][9]))
        prec.append(float(results[i][13]))
    '''    
    for i in range(len(resultsSP)):
        speedSP.append(float(resultsSP[i][6]))
        mSP.append(float(resultsSP[i][2]))
        dSP.append(float(resultsSP[i][3]))
        hSP.append(float(resultsSP[i][4]))
    for i in range(len(resultsH)):
        speedH.append(float(resultsH[i][6]))
        mH.append(float(resultsH[i][2]))
        dH.append(float(resultsH[i][3]))
        hH.append(float(resultsH[i][4]))
        
    '''
    #months['hourly_%d' % y] = np.asarray(m).copy()
    #only take summer months
    #iseason = np.where(abs(np.asarray(m) - 7.) <= 1.)[0]
    #iseason1 = np.where(np.asarray(m) == 12.)[0]
    iseason = np.where(np.asarray(m) <= 2.)[0]
    #iseason = np.asarray((iseason1).tolist() + (iseason2).tolist())
    
    wind_dir['hourly_%d' % y] = np.asarray(direction)[iseason]
    wind_speed['hourly_%d' % y] = np.asarray(speed)[iseason]
    #wind_speedSP['hourly_%d' % y] = np.asarray(speedSP)
    #wind_speedH['hourly_%d' % y] = np.asarray(speedH)
    temperature['hourly_%d' % y] = np.asarray(temp)[iseason]
    precipitation['hourly_%d' % y] = np.asarray(prec)[iseason]
    months['hourly_%d' % y] = np.asarray(m)[:]
    #monthsSP['hourly_%d' % y] = np.asarray(mSP)
    #monthsH['hourly_%d' % y] = np.asarray(mH)
    day['hourly_%d' % y] = np.asarray(d)[iseason]
    #daySP['hourly_%d' % y] = np.asarray(dSP)
    #dayH['hourly_%d' % y] = np.asarray(dH)
    hour['hourly_%d' % y] = np.asarray(h)[iseason]
    #hourSP['hourly_%d' % y] = np.asarray(hSP)
    #hourH['hourly_%d' % y] = np.asarray(hH)

    #time['hourly_%d' % y] = np.arange(1., 13., 12./len(wind_speed['hourly_%d' % y]))
    

    

In [5]:
wind_speedUM = dict()
#wind_speedSPUM = dict()
#wind_speedHUM = dict()
for y in YEAR:
    wind_speedUM['hourly_%d' % y] = wind_speed['hourly_%d' % y]
    #wind_speedSPUM['hourly_%d' % y] = wind_speedSP['hourly_%d' % y]
    #wind_speedHUM['hourly_%d' % y] = wind_speedH['hourly_%d' % y]

In [17]:
for y in YEAR:
    wind_speed['hourly_%d' % y] = ma.masked_equal(wind_speed['hourly_%d' % y], -99.9)
    months['hourly_%d' % 2018] = ma.masked_equal(wind_speed['hourly_%d' % y], -99.9)
    #wind_speedSP['hourly_%d' % y] = ma.masked_equal(wind_speedSP['hourly_%d' % y], -999.9)
    #wind_speedSP['hourly_%d' % y] = ma.masked_equal(wind_speedSP['hourly_%d' % y], -99.9)
    #wind_speedH['hourly_%d' % y] = ma.masked_equal(wind_speedH['hourly_%d' % y], -999.9)
    #wind_speedH['hourly_%d' % y] = ma.masked_equal(wind_speedH['hourly_%d' % y], -99.9)
    wind_dir['hourly_%d' % y] = ma.masked_equal(wind_dir['hourly_%d' % y], -999.)
    temperature['hourly_%d' % y] = ma.masked_equal(temperature['hourly_%d' % y], -999.9)
    precipitation['hourly_%d' % y] = ma.masked_equal(precipitation['hourly_%d' % y], -999.9)

In [18]:
for y in YEAR:
    for i in range(len(temperature['hourly_%d' % y])):
        temperature['hourly_%d' % y][i] = temperature['hourly_%d' % y][i]*9./5. + 32.

In [19]:
ws_good = dict()
prec_good = dict()
#ws_goodSP = dict()
#ws_goodH = dict()

#create arrays with missing elements deleted
for y in YEAR:
    
    w = wind_speed['hourly_%d' % y]
    p = precipitation['hourly_%d' % y]
    #wSP = wind_speedSP['hourly_%d' % y]
    #wH = wind_speedH['hourly_%d' % y]

    igood = np.where(w >= 0.)[0]
    igoodp = np.where(p >= 0.)[0]
    #igoodSP = np.where(wSP >= 0.)[0]
    #igoodH = np.where(wH >= 0.)[0]

    ws_good['hourly_%d' % y] = w[igood]
    prec_good['hourly_%d' % y] = p[igoodp]
    #ws_goodSP['hourly_%d' % y] = wSP[igoodSP]
    #ws_goodH['hourly_%d' % y] = wH[igoodH]
    

In [21]:
#create cumulative distribution:

base = defaultdict(dict)
cumu = defaultdict(dict)
cumu_I = defaultdict(dict)

bins = 50
scale = 1#2.23694
figure()
for y in YEAR:
    
    ws = ws_good['hourly_%d' % y]
    values, base[y] = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu[y] = cumulative/cumulative.max()
    cumu_I[y] = (1. - cumu[y])
    
    if y == 2018:
        plot(base[y][:-1]*scale, (cumu_I[y])*100., 'k-', linewidth = 3, label ='%d' % y)
    else:    
        plot(base[y][:-1]*scale, (cumu_I[y])*100., '--', linewidth = 2, label ='%d' % y)
    

plt.title('January-February: Prob(Hourly Average Wind Speed $>$ x)')
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('Cumulative Distribution (%)')
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)
#plt.legend([c_15, c_14, c_13, c_12, c_11, c_10, c_09], ['2015', '2014', '2013', '2012', '2011', '2010', '2009'], loc = 'lower right')
legend()
#plt.xlim([0,ws.max()])
plt.ylim([0, 105])
#plt.savefig('../meteorology/plots/year_to_year_comparisons/wind_speed_hourly_average_1-cumulative_dist.png')
plt.show()

/home/arikaplan/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:1288: UserWarning: findfont: Font family ['normal'] not found. Falling back to Bitstream Vera Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [89]:
figure()
plot(prec_good['hourly_%d' % 2017])

In [88]:
#create cumulative distribution:

base = defaultdict(dict)
cumu = defaultdict(dict)
cumu_I = defaultdict(dict)

bins = 50
scale = 1#2.23694

for y in YEAR:
    
    ws = prec_good['hourly_%d' % y]
    values, base[y] = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu[y] = cumulative/cumulative.max()
    cumu_I[y] = (1. - cumu[y])
    
    if y == 2018:
        plot(base[y][:-1]*scale, (cumu_I[y])*100., 'k-', linewidth = 2, label ='%d' % y)
    else:    
        plot(base[y][:-1]*scale, (cumu_I[y])*100., '--', linewidth = 2, label ='%d' % y)
    

plt.title('June-August: Prob(Precipitation per Hour $>$ x)')
plt.xlabel('Precipitation (mm/hour)')
plt.ylabel('Cumulative Distribution (%)')
#plt.legend([c_15, c_14, c_13, c_12, c_11, c_10, c_09], ['2015', '2014', '2013', '2012', '2011', '2010', '2009'], loc = 'lower right')
legend()
#plt.xlim([0,ws.max()])
plt.ylim([0, 105])
#plt.savefig('../meteorology/plots/year_to_year_comparisons/wind_speed_hourly_average_1-cumulative_dist.png')
plt.show()

In [99]:
print YEAR

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


In [100]:
#hourly average for each year
for y in [2009]:
    plt.figure(figsize=(10,5))
    plt.plot(permonth['hourly_%d' % y], wind_speed['hourly_%d' % y], 'b-', linewidth = 1)
    plt.title('%d Hourly Average Wind Speed at Summit Station' % y)
    plt.ylabel('Wind Speed (m/s)')
    plt.xlabel('Month')
    #maximum = wind_speed['hourly_%d' % y].max()
    #plt.ylim([0, maximum + .05*maximum])
    #plt.savefig('../meteorology/plots/%d/%d_wind_speed_hourly_average.png' % (y, y))
    plt.show()
    #plt.close('all')

ValueError: x and y must have same first dimension, but have shapes (4416L,) and (2208L,)

In [9]:
#look month by month

In [9]:
perday = dict()
maxPerDay = dict()

for y in YEAR:
    d = []
    newMonth = []
    start = months['hourly_%d' % y][0]
    end = months['hourly_%d' % y][-1]
    for i in range(len(temperature['hourly_%d' % y])):
    #for i in range(len(wind_speed['hourly_%d' % y])):
        #t = months['hourly_%d' % y][i] + ((day['hourly_%d' % y][i] -1)/ numDays[int(months['hourly_%d' % y][i]) - 1]) + (hour['hourly_%d' % y][i] / numDays[int(months['hourly_%d' % y][i]) - 1] / 24)
        t = (day['hourly_%d' % y][i] -1) + (hour['hourly_%d' % y][i] / 24)
        d.append(t)
        if i > 0 and months['hourly_%d' % y][i] != months['hourly_%d' % y][i-1]:
            newMonth.append(i)
            

    perday['hourly_%d_%d' % (end, y)] = d[newMonth[-1]:]
    #wind_speed['hourly_%d_%d' % (end, y)] = wind_speed['hourly_%d' % y][newMonth[-1]:]
    #wind_speedUM['hourly_%d_%d' % (end, y)] = wind_speedUM['hourly_%d' % y][newMonth[-1]:]
    temperature['hourly_%d_%d' % (end, y)] = temperature['hourly_%d' % y][newMonth[-1]:]
    #wind_dir['hourly_%d_%d' % (end, y)] = wind_dir['hourly_%d' % y][newMonth[-1]:]

    for c in range(len(newMonth)):
        if c == 0:
            perday['hourly_%d_%d' % (start + c, y)] = d[0:newMonth[c]]
            #wind_speed['hourly_%d_%d' % (start + c, y)] = wind_speed['hourly_%d' % y][0:newMonth[c]]
            #wind_speedUM['hourly_%d_%d' % (start + c, y)] = wind_speedUM['hourly_%d' % y][0:newMonth[c]]
            temperature['hourly_%d_%d' % (start + c, y)] = temperature['hourly_%d' % y][0:newMonth[c]]
            #wind_dir['hourly_%d_%d' % (start + c, y)] = wind_dir['hourly_%d' % y][0:newMonth[c]]
            
        else:
            perday['hourly_%d_%d' % (start + c, y)] = d[newMonth[c-1]:newMonth[c]]
            #wind_speed['hourly_%d_%d' % (start + c, y)] = wind_speed['hourly_%d' % y][newMonth[c-1]:newMonth[c]]
            #wind_speedUM['hourly_%d_%d' % (start + c, y)] = wind_speedUM['hourly_%d' % y][newMonth[c-1]:newMonth[c]]
            temperature['hourly_%d_%d' % (start + c, y)] = temperature['hourly_%d' % y][newMonth[c-1]:newMonth[c]]
            #wind_dir['hourly_%d_%d' % (start + c, y)] = wind_dir['hourly_%d' % y][newMonth[c-1]:newMonth[c]]

In [363]:
perdaySP = dict()
maxPerDaySP = dict()

for y in YEAR:
    d = []
    newMonth = []
    start = monthsSP['hourly_%d' % y][0]
    end = monthsSP['hourly_%d' % y][-1]
    for i in range(len(wind_speedSP['hourly_%d' % y])):
        #t = months['hourly_%d' % y][i] + ((day['hourly_%d' % y][i] -1)/ numDays[int(months['hourly_%d' % y][i]) - 1]) + (hour['hourly_%d' % y][i] / numDays[int(months['hourly_%d' % y][i]) - 1] / 24)
        t = (daySP['hourly_%d' % y][i] -1) + (hourSP['hourly_%d' % y][i] / 24)
        d.append(t)
        if i > 0 and monthsSP['hourly_%d' % y][i] != monthsSP['hourly_%d' % y][i-1]:
            newMonth.append(i)
            

    perdaySP['hourly_%d_%d' % (end, y)] = d[newMonth[-1]:]
    wind_speedSP['hourly_%d_%d' % (end, y)] = wind_speedSP['hourly_%d' % y][newMonth[-1]:]
    wind_speedSPUM['hourly_%d_%d' % (end, y)] = wind_speedSPUM['hourly_%d' % y][newMonth[-1]:]

    for c in range(len(newMonth)):
        if c == 0:
            perdaySP['hourly_%d_%d' % (start + c, y)] = d[0:newMonth[c]]
            wind_speedSP['hourly_%d_%d' % (start + c, y)] = wind_speedSP['hourly_%d' % y][0:newMonth[c]]
            wind_speedSPUM['hourly_%d_%d' % (start + c, y)] = wind_speedSPUM['hourly_%d' % y][0:newMonth[c]]
            
        else:
            perdaySP['hourly_%d_%d' % (start + c, y)] = d[newMonth[c-1]:newMonth[c]]
            wind_speedSP['hourly_%d_%d' % (start + c, y)] = wind_speedSP['hourly_%d' % y][newMonth[c-1]:newMonth[c]]
            wind_speedSPUM['hourly_%d_%d' % (start + c, y)] = wind_speedSPUM['hourly_%d' % y][newMonth[c-1]:newMonth[c]]

In [361]:
perdayH = dict()
maxPerDayH = dict()

for y in YEAR:
    d = []
    newMonth = []
    start = monthsH['hourly_%d' % y][0]
    end = monthsH['hourly_%d' % y][-1]
    for i in range(len(wind_speedH['hourly_%d' % y])):
        #t = months['hourly_%d' % y][i] + ((day['hourly_%d' % y][i] -1)/ numDays[int(months['hourly_%d' % y][i]) - 1]) + (hour['hourly_%d' % y][i] / numDays[int(months['hourly_%d' % y][i]) - 1] / 24)
        t = (dayH['hourly_%d' % y][i] -1) + (hourH['hourly_%d' % y][i] / 24)
        d.append(t)
        if i > 0 and monthsH['hourly_%d' % y][i] != monthsH['hourly_%d' % y][i-1]:
            newMonth.append(i)
            

    perdayH['hourly_%d_%d' % (end, y)] = d[newMonth[-1]:]
    wind_speedH['hourly_%d_%d' % (end, y)] = wind_speedH['hourly_%d' % y][newMonth[-1]:]
    wind_speedHUM['hourly_%d_%d' % (end, y)] = wind_speedHUM['hourly_%d' % y][newMonth[-1]:]

    for c in range(len(newMonth)):
        if c == 0:
            perdayH['hourly_%d_%d' % (start + c, y)] = d[0:newMonth[c]]
            wind_speedH['hourly_%d_%d' % (start + c, y)] = wind_speedH['hourly_%d' % y][0:newMonth[c]]
            wind_speedHUM['hourly_%d_%d' % (start + c, y)] = wind_speedHUM['hourly_%d' % y][0:newMonth[c]]
            
        else:
            perdayH['hourly_%d_%d' % (start + c, y)] = d[newMonth[c-1]:newMonth[c]]
            wind_speedH['hourly_%d_%d' % (start + c, y)] = wind_speedH['hourly_%d' % y][newMonth[c-1]:newMonth[c]]
            wind_speedHUM['hourly_%d_%d' % (start + c, y)] = wind_speedHUM['hourly_%d' % y][newMonth[c-1]:newMonth[c]]

In [269]:
wind_speedB = dict()
wind_speedB['monthly_2015'] = [3.8, 4.8, 3.5, 4.2, 3.6, 3.3, 2.8, 2.8, 2.7, 3.6, 4.8, 5.7]
wind_speedB['monthly_2014'] = [4.2, 5.4, 5.1, 4.4, 4.1, 3.4, 2.9, 3.0, 3.4, 4.0, 4.5, 6.]
wind_speedB['monthly_2013'] = [4.965, 5.041, 4.654, 4.649, 3.744, 3.271, 2.723, 2.782, 3.76, 3.585, 3.479, 4.774]
wind_speedB['monthly_2012'] = [5.051, 5.235, 6.216, 4.853, 4.027, 3.672, 2.796, 2.655, 2.547, 3.894, 4.432, 5.835]
wind_speedB['monthly_2011'] = [4.561, 6.134, 5.986, 5.524, 4.845, 3.829, 2.631, 2.649, 2.391, 3.933, 4.933, 4.202]
wind_speedB['monthly_2010'] = [4.801, 4.764, 5.309, -99.9, -99.9, -99.9, 2.754, 2.948, 2.822, 4.066, 5.25, 5.85]
wind_speedB['monthly_2009'] = [4.682, 4.784, 5.396, 4.717, 3.545, 3.027, 2.801, 2.867, 2.707, 3.627, 4.388, 5.513]

for y in YEAR:
    wind_speedB['monthly_%d' % y] = np.asarray(wind_speedB['monthly_%d' % y])
    wind_speedB['monthly_%d' % y] = ma.masked_equal(wind_speedB['monthly_%d' % y], -99.9)


In [257]:
wind_speedB_max = dict()
wind_speedB_max['monthly_2015'] = [17.77, 28.45, 16.92, 24.89, 17.31, 15.35, 13.33, 14.14, 18.42, 20.28, 23.52, 24.37]
wind_speedB_max['monthly_2014'] = [24.53, 24.66, 23.78, 23.78, 22.8, 16.98, 16.85, 17.47, 21.43, 19.79, 25.61, 25.51]
wind_speedB_max['monthly_2013'] = [23.52, 31.1, 22.31, 33.75, 18.72, 16.69, 17.77, 14.73, 19.7, 20.68, 16.33, 23.39]
wind_speedB_max['monthly_2012'] = [27.37, 25.9, 25.41, 20.02, 21.52, 19.08, 15.48, 15.32, 13.36, 25.7, 22.18, 24.01]
wind_speedB_max['monthly_2011'] = [20.94, 25.41, 23.39, 23.03, 23.06, 20.74, 14.11, 13.88, 14.96, 20.87, 27.01, 25.02]
wind_speedB_max['monthly_2010'] = [25.22, 20.09, 30.02, -99.9, -99.9, -99.9, 16.66, 16.1, 16.72, 23.62, 23.52, 26.78]
wind_speedB_max['monthly_2009'] = [24.53, 21.3, 23.52, 26.39, 19.63, 14.37, 14.86, 15.97, 21.17, 27.7, 22.05, 28.09]

for y in YEAR:
    wind_speedB_max['monthly_%d' % y] = np.asarray(wind_speedB_max['monthly_%d' % y])
    wind_speedB_max['monthly_%d' % y] = ma.masked_equal(wind_speedB_max['monthly_%d' % y], -99.9)


In [268]:
y = 2015
print wind_speedB['monthly_%d' % y]

[ 17.77  28.45  16.92  24.89  17.31  15.35  13.33  14.14  18.42  20.28
  23.52  24.37]


In [271]:
#monthly site comparison
for y in YEAR:
    mavgG = []
    mavgSP = []
    mavgH = []
    mB = wind_speedB['monthly_%d' % y]
    #mavgWM = [9.6, 12.2, 8.8, 10.1, 7.4, 8.7, 7.2, 8.3, 7.4, 8.4, 11.9, 14.5]
    for m in range(1,13):
        mavgG.append(wind_speed['minute_%d_%d' % (m, y)].mean())
        mavgSP.append(wind_speedSP['minute_%d_%d' % (m,y)].mean())
        mavgH.append(wind_speedH['minute_%d_%d' % (m,y)].mean())

    mavgG = np.asarray(mavgG)
    #mavgB = np.asarray(wind_speedB_max['monthly_%d' % y])
    mavgSP = np.asarray(mavgSP)
    mavgH = np.asarray(mavgH)
    #mavgWM = np.asarray(mavgWM)

    G, = plt.plot(np.arange(1,13), mavgG*scale, 'g-', linewidth = 2)
    B, = plt.plot(np.arange(1,13), mB*scale, 'k-', linewidth = 2)
    SP, = plt.plot(np.arange(1,13), mavgSP*scale, 'r-', linewidth = 2)
    H, = plt.plot(np.arange(1,13), mavgH*scale, 'b-', linewidth = 2)
    #WM, = plt.plot(np.arange(1,13), mavgWM*scale, 'g-', linewidth = 2)


    plt.legend([G, B, SP, H], ['Greenland', 'Barcroft', 'South Pole', 'Mauna Loa'], loc = 'upper right')
    #plt.legend([B, SP, WM], ['Barcroft', 'South Pole', 'White Mountain Summit'], loc = 'upper left')
    plt.ylabel('Wind Speed (knots)')
    plt.xlabel('Month')
    plt.title('%d Wind Speed Monthly Mean' % y)
    plt.xlim([1, 13])
    plt.ylim([0, 30])
    plt.savefig('../meteorology/plots/%d/%d_monthly_wind_speed_average_site_comparison.png' % (y,y))
    plt.close('all')
    #plt.show()

In [ ]:
print ma.count_masked()

In [281]:
y = 2014
ws = dailyMean_y['%d' % y]*scale
print ma.count_masked(ws)
#bins = round(ws.max())*4.
values, baseG = np.histogram(ws, bins = bins, density=True)
cumulative = np.cumsum(values)
cumu_G = cumulative/cumulative.max()

#print ws
#print values
#print cumulative[0:6]
#print cumu_G[0:6]

0


In [ ]:
y = 2014


In [391]:
#create cumulative distribution for different sites:

for y in YEAR:
    bins = 20
    ws = dailyMean_y['%d' % y]*scale
    #bins = round(ws.max())*4.
    values, baseG = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_G = cumulative/cumulative.max()

    ws = dailyMeanSP_y['%d' % y]*scale
    #bins = round(ws.max())*4.
    values, baseSP = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_SP = cumulative/cumulative.max()

    ws = dailyMeanH_y['%d' % y]*scale
    #bins = round(ws.max())*4.
    values, baseH = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_H = cumulative/cumulative.max()

    ws = dailyMeanB_good['%d' % y]*scale
    #bins = round(ws.max())*4.
    values, baseB = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_B = cumulative/cumulative.max()


    #plt.plot(base[:-1], values, 'g-')
    c_G, = plt.plot(baseG[:-1], (cumu_G)*100., 'g-', linewidth = 2)
    c_B, = plt.plot(baseB[:-1], (cumu_B)*100., 'k-', linewidth = 2)
    c_SP, = plt.plot(baseSP[:-1], (cumu_SP)*100., 'r-', linewidth = 2)
    c_H, = plt.plot(baseH[:-1], (cumu_H)*100., 'b-', linewidth = 2)

    plt.title('%d Prob(Daily Mean Wind Speed $\leq$ x)' % y)
    plt.xlabel('Wind Speed (knots)')
    plt.ylabel('Cumulative Distribution (%)')
    plt.legend([c_G, c_B, c_SP, c_H], ['Greenland', 'Barcroft', 'South Pole', 'Hawaii'], loc = 'lower right')

    #plt.xlim([0,ws.max()])
    plt.ylim([0, 105])
    plt.savefig('../meteorology/plots/%d/%d_wind_speed_mean_cumulative_dist.png' % (y,y))
    #plt.show()
    plt.close('all')

In [15]:
#create arrays with missing elements deleted
for y in YEAR:
    for m in range(1,13):
        w = wind_speed['hourly_%d_%d' % (m,y)]

        igood = np.where(w >= 0.)[0]

        ws_good['hourly_%d_%d' % (m,y)] = w[igood]*1.94384


In [10]:
#find daily average and standard deviation for each month
dailyMean = defaultdict(list)
dailySTD = defaultdict(list)
#dailyMean_y = defaultdict(list)
#dailyMeanUM = defaultdict(list)

for y in YEAR:
    #dailyMean_y['%d' % y] = np.empty(0)
    for c in range(1, 13):
        newDay = []
        for i in range(len(temperature['hourly_%d_%d' % (c, y)])):
            if i > 0 and day['hourly_%d' % y][i] != day['hourly_%d' % y][i-1]:
                newDay.append(i)

        for d in range(len(newDay)):
            if d == 0:
                dailyMean['%d_%d' % (c, y)].append(temperature['hourly_%d_%d' % (c, y)][0:newDay[d]].mean())
                #dailyMeanUM['%d_%d' % (c, y)].append(wind_speedUM['hourly_%d_%d' % (c, y)][0:newDay[d]].mean())
                dailySTD['%d_%d' % (c, y)].append(temperature['hourly_%d_%d' % (c, y)][0:newDay[d]].std())      
            else:
                dailyMean['%d_%d' % (c, y)].append(temperature['hourly_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].mean())
                #dailyMeanUM['%d_%d' % (c, y)].append(wind_speedUM['hourly_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].mean())
                dailySTD['%d_%d' % (c, y)].append(temperature['hourly_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].std())  

        dailyMean['%d_%d' % (c, y)].append(temperature['hourly_%d_%d' % (c, y)][newDay[-1]:].mean())
        #dailyMeanUM['%d_%d' % (c, y)].append(wind_speedUM['hourly_%d_%d' % (c, y)][newDay[-1]:].mean())
        dailySTD['%d_%d' % (c, y)].append(temperature['hourly_%d_%d' % (c, y)][newDay[-1]:].std()) 
        
        #dailyMean_y['%d' % y] = np.hstack([dailyMean_y['%d' % y], dailyMeanUM['%d_%d' % (c, y)]])
    
    #dailyMean_y['%d' % y] = ma.masked_where(dailyMean_y['%d' % y] < 0., dailyMean_y['%d' % y])

In [387]:
#create arrays with missing elements deleted
for y in YEAR:
    w = dailyMean_y['%d' % y]

    igood = np.where(w >= 0.)[0]

    dailyMean_y['%d' % y] = w[igood]


In [381]:
#find daily average and standard deviation for each month
dailyMeanSP = defaultdict(list)
dailyMeanSPUM = defaultdict(list)
dailyMeanSP_y = defaultdict(list)


for y in YEAR:
    dailyMeanSP_y['%d' % y] = np.empty(0)
    for c in range(1, 13):
        newDay = []
        for i in range(len(wind_speedSP['hourly_%d_%d' % (c, y)])):
            if i > 0 and daySP['hourly_%d' % y][i] != daySP['hourly_%d' % y][i-1]:
                newDay.append(i)

        for d in range(len(newDay)):
            if d == 0:
                dailyMeanSP['%d_%d' % (c, y)].append(wind_speedSP['hourly_%d_%d' % (c, y)][0:newDay[d]].mean())
                dailyMeanSPUM['%d_%d' % (c, y)].append(wind_speedSPUM['hourly_%d_%d' % (c, y)][0:newDay[d]].mean())
            else:
                dailyMeanSP['%d_%d' % (c, y)].append(wind_speedSP['hourly_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].mean())
                dailyMeanSPUM['%d_%d' % (c, y)].append(wind_speedSPUM['hourly_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].mean())

        dailyMeanSP['%d_%d' % (c, y)].append(wind_speedSP['hourly_%d_%d' % (c, y)][newDay[-1]:].mean())
        dailyMeanSPUM['%d_%d' % (c, y)].append(wind_speedSPUM['hourly_%d_%d' % (c, y)][newDay[-1]:].mean())
        
        dailyMeanSP_y['%d' % y] = np.hstack([dailyMeanSP_y['%d' % y], dailyMeanSPUM['%d_%d' % (c, y)]])
    dailyMeanSP_y['%d' % y] = ma.masked_where(dailyMeanSP_y['%d' % y] < 0., dailyMeanSP_y['%d' % y])

In [388]:
#create arrays with missing elements deleted
for y in YEAR:
    w = dailyMeanSP_y['%d' % y]

    igood = np.where(w >= 0.)[0]

    dailyMeanSP_y['%d' % y] = w[igood]


In [384]:
#find daily average and standard deviation for each month
dailyMeanH = defaultdict(list)
dailyMeanHUM = defaultdict(list)
dailyMeanH_y = defaultdict(list)

for y in YEAR:
    dailyMeanH_y['%d' % y] = np.empty(0)
    for c in range(1, 13):
        newDay = []
        for i in range(len(wind_speedH['hourly_%d_%d' % (c, y)])):
            if i > 0 and dayH['hourly_%d' % y][i] != dayH['hourly_%d' % y][i-1]:
                newDay.append(i)

        for d in range(len(newDay)):
            if d == 0:
                dailyMeanH['%d_%d' % (c, y)].append(wind_speedH['hourly_%d_%d' % (c, y)][0:newDay[d]].mean())
                dailyMeanHUM['%d_%d' % (c, y)].append(wind_speedHUM['hourly_%d_%d' % (c, y)][0:newDay[d]].mean())
            else:
                dailyMeanH['%d_%d' % (c, y)].append(wind_speedH['hourly_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].mean())
                dailyMeanHUM['%d_%d' % (c, y)].append(wind_speedHUM['hourly_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].mean())

        dailyMeanH['%d_%d' % (c, y)].append(wind_speedH['hourly_%d_%d' % (c, y)][newDay[-1]:].mean())
        dailyMeanHUM['%d_%d' % (c, y)].append(wind_speedHUM['hourly_%d_%d' % (c, y)][newDay[-1]:].mean())
        
        dailyMeanH_y['%d' % y] = np.hstack([dailyMeanH_y['%d' % y], dailyMeanHUM['%d_%d' % (c, y)]])
    dailyMeanH_y['%d' % y] = ma.masked_where(dailyMeanH_y['%d' % y] < 0., dailyMeanH_y['%d' % y])

In [389]:
#create arrays with missing elements deleted
for y in YEAR:
    w = dailyMeanH_y['%d' % y]

    igood = np.where(w >= 0.)[0]

    dailyMeanH_y['%d' % y] = w[igood]


In [69]:
#load barcroft data
dailyMeanB = dict()
dailyMaxB = dict()
dailyMeanB_good = dict()
dailyMaxB_good = dict()
for y in YEAR:
#for m in range(1,13):

    file1 = '../meteorology/data/barcroft/barcroft_%d.txt' % (y)

    results = []
    with open(file1) as inputfile:
        for line in inputfile:
            results.append(line.strip().split())

    speed = []
    speedMax = []

    for i in range(len(results)):
        speed.append(float(results[i][4]))
        speedMax.append(float(results[i][6]))

    dailyMeanB['%d' % y] = np.asarray(speed)
    w = dailyMeanB['%d' % y]
    igood = np.where(w >= 0.)[0]
    dailyMeanB_good['%d' % y] = w[igood]

    dailyMeanB['%d' % y] = ma.masked_equal(dailyMeanB['%d' % y], -9999)
    
    dailyMaxB['%d' % y] = np.asarray(speedMax)
    w = dailyMaxB['%d' % y]
    igood = np.where(w >= 0.)[0]
    dailyMaxB_good['%d' % y] = w[igood]

    dailyMaxB['%d' % y] = ma.masked_equal(dailyMaxB['%d' % y], -9999)



In [234]:
for y in YEAR:
    print dailyMaxB['%d' % y].min()

0.0
5.07
3.75
5.42
3.27
4.51
3.82


In [129]:
#load barcroft data
dailyMeanB_month = dict()
dailyMaxB_month = dict()
dailyMeanB_month_good = dict()
dailyMaxB_month_good = dict()
for y in [2015]:
    for m in range(1,13):
        if m < 10:
            M = '0%d' % m
        else:
            M = '%d' % m
        file1 = '../meteorology/data/barcroft/%d/month/barcroft_%s_%d.txt' % (y, M, y)

        results = []
        with open(file1) as inputfile:
            for line in inputfile:
                results.append(line.strip().split())

        speed = []
        speedMax = []

        for i in range(len(results)):
            speed.append(float(results[i][3]))
            speedMax.append(float(results[i][5]))

        dailyMeanB_month['%d_%d' % (m, y)] = np.asarray(speed)
        w = dailyMeanB_month['%d_%d' % (m, y)]
        igood = np.where(w >= 0.)[0]
        dailyMeanB_month_good['%d_%d' % (m, y)] = w[igood]

        dailyMeanB_month['%d_%d' % (m, y)] = ma.masked_equal(dailyMeanB_month['%d_%d' % (m, y)], -99.9)

        dailyMaxB_month['%d_%d' % (m, y)] = np.asarray(speedMax)
        w = dailyMaxB_month['%d_%d' % (m, y)]
        igood = np.where(w >= 0.)[0]
        dailyMaxB_month_good['%d_%d' % (m, y)] = w[igood]

        dailyMaxB_month['%d_%d' % (m, y)] = ma.masked_equal(dailyMaxB_month['%d_%d' % (m, y)], -99.9)



In [ ]:
#create arrays with missing elements deleted
for y in YEAR:
    for m in range(1,13):
        w = wind_speed['hourly_%d_%d' % (m,y)]
        h = humidity['hourly_%d_%d' % (m,y)]

        igood = np.where(w >= 0.)[0]
        igoodh = np.where(h >= 0.)[0]

        ws_good['hourly_%d_%d' % (m,y)] = w[igood]*1.94384
        hum_good['hourly_%d_%d' % (m,y)] = h[igood]

In [66]:
#month by month site comparison

In [166]:
print len(days)
print len(dailyMaxSP['%d_%d' % (m, y)])
print m, y


30
29
11 2015


In [167]:
y = 2015
for m in range(1,13):
    plt.figure(figsize=(10,5))

    davgG = np.asarray(dailyMax['%d_%d' % (m, y)])
    davgB = dailyMaxB_month['%d_%d' % (m, y)]
    davgSP = np.asarray(dailyMaxSP['%d_%d' % (m, y)])
    davgH = np.asarray(dailyMaxH['%d_%d' % (m, y)])

    days = np.arange(1, numDays[m-1]+1)
    if m == 11 and y == 2015:
        daysSP = np.arange(len(davgSP))
    else: 
        daysSP = days
    G, = plt.plot(days, davgG*scale, 'g-', linewidth = 2)
    B, = plt.plot(days, davgB*scale, 'k-', linewidth = 2)
    SP, = plt.plot(daysSP, davgSP*scale, 'r-', linewidth = 2)
    H, = plt.plot(days, davgH*scale, 'b-', linewidth = 2)
    #WM, = plt.plot(np.arange(1,13), mavgWM*scale, 'g-', linewidth = 2)


    plt.legend([G, B, SP, H], ['Greenland', 'Barcroft', 'South Pole', 'Mauna Loa'], loc = 'upper right')
    #plt.legend([B, SP, WM], ['Barcroft', 'South Pole', 'White Mountain Summit'], loc = 'upper left')
    plt.ylabel('Wind Speed (knots)')
    plt.xlabel('day')
    plt.title('%s %d Wind Speed Daily Maximums' % (month[m-1], y))
    #plt.xlim([1, 13])
    #plt.ylim([0, 35])
    plt.savefig('../meteorology/plots/2015/%s/%s_%d_daily_wind_speed_max_site_comparison.png' % (month[m-1].lower(), month[m-1].lower(), y))
    plt.close('all')
    #plt.show()

In [11]:
#plot average and standard deviation for each day in each month
scale = 1.
for c in range(6, 9):
    plt.figure(figsize=(10,5))
    
    y = 2015
    y2015, = plt.plot(np.arange(len(dailyMean['%d_%d' % (c, y)])), np.asarray(dailyMean['%d_%d' % (c, y)])*scale, 'b-', linewidth = 2)
    y = 2014
    y2014, = plt.plot(np.arange(len(dailyMean['%d_%d' % (c, y)])), np.asarray(dailyMean['%d_%d' % (c, y)])*scale, 'r-', linewidth = 2)
    y = 2013
    y2013, = plt.plot(np.arange(len(dailyMean['%d_%d' % (c, y)])), np.asarray(dailyMean['%d_%d' % (c, y)])*scale, 'g-', linewidth = 2)
    y = 2012
    y2012, = plt.plot(np.arange(len(dailyMean['%d_%d' % (c, y)])), np.asarray(dailyMean['%d_%d' % (c, y)])*scale, 'y-', linewidth = 2)

    plt.legend([y2015, y2014, y2013, y2012], ['2015', '2014', '2013', '2012'], loc = 'upper right')

    plt.title('%s Daily Temperature Average' % month[c-1])
    plt.xlabel('Day')
    #plt.ylabel('wind speed (%s)' % units)
    plt.ylabel('degrees (Fahrenheit)')
    plt.ylim([-30, 30])
    #plt.show()
    plt.savefig('../meteorology/plots/year_to_year_comparisons/%s/%s_2012-2015_daily_temperature_average.png' % 
                (month[c-1].lower(), month[c-1].lower()))
    plt.close('all')
    
    plt.figure(figsize=(10,5))
    y = 2015
    y2015, = plt.plot(np.arange(len(dailySTD['%d_%d' % (c, y)])), np.asarray(dailySTD['%d_%d' % (c, y)])*scale, 'b-', linewidth = 2)
    y = 2014
    y2014, = plt.plot(np.arange(len(dailySTD['%d_%d' % (c, y)])), np.asarray(dailySTD['%d_%d' % (c, y)])*scale, 'r-', linewidth = 2)
    y = 2013
    y2013, = plt.plot(np.arange(len(dailySTD['%d_%d' % (c, y)])), np.asarray(dailySTD['%d_%d' % (c, y)])*scale, 'g-', linewidth = 2)
    y = 2012
    y2012, = plt.plot(np.arange(len(dailySTD['%d_%d' % (c, y)])), np.asarray(dailySTD['%d_%d' % (c, y)])*scale, 'y-', linewidth = 2)

    plt.legend([y2015, y2014, y2013, y2012], ['2015', '2014', '2013', '2012'], loc = 'upper right')

    plt.title('%s Daily Temperature Standard Deviation' % month[c-1])
    plt.xlabel('Day')
    #plt.ylabel('wind speed (%s)' % units)
    plt.ylabel('degrees (Fahrenheit)')
    plt.ylim([0, 20])
    #plt.show()
    plt.savefig('../meteorology/plots/year_to_year_comparisons/%s/%s_2012-2015_daily_temperature_std.png' % 
                (month[c-1].lower(), month[c-1].lower()))
    plt.close('all')

In [12]:
#plot hourly averages for each month in each year
for y in YEAR:
    start = int(months['hourly_%d' % y][0])
    end = int(months['hourly_%d' % y][-1])
    #for c in range(start,end+1):
    for c in range(6,8+1):
        plt.plot(perday['hourly_%d_%d' % (c, y)], temperature['hourly_%d_%d' % (c, y)], 'b-', linewidth = 1)
        plt.title('%s %d Hourly Average Temperature at Summit Station' % (month[c-1], y))
        plt.ylabel('Degrees (Fahrenheit)')
        plt.xlabel('Day')
        plt.ylim([-40, 40])
        #plt.xlim([1, 744 * 12./len(wind_speed)])
        plt.savefig('../meteorology/plots/%d/%s/%s_%d_temperature_hourly_average.png' % (y, month[c-1].lower(), month[c-1].lower(), y))
        #plt.show()
        plt.close('all')

In [ ]:
#histogram plots

In [165]:
#create cumulative distribution:
for m in range(1,13):
    bins = 10
    
    y = 2009
    ws = ws_good['hourly_%d_%d' % (m,y)]
    #bins = round(ws.max())
    values, base09 = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_2009 = cumulative/cumulative.max()

    y = 2010
    ws = ws_good['hourly_%d_%d' % (m,y)]
    #bins = round(ws.max())
    values, base10 = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_2010 = cumulative/cumulative.max()

    y = 2011
    ws = ws_good['hourly_%d_%d' % (m,y)]
    #bins = round(ws.max())
    values, base11 = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_2011 = cumulative/cumulative.max()

    y = 2012
    ws = ws_good['hourly_%d_%d' % (m,y)]
    #bins = round(ws.max())
    values, base12 = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_2012 = cumulative/cumulative.max()


    y = 2013
    ws = ws_good['hourly_%d_%d' % (m,y)]
    #bins = round(ws.max())
    values, base13 = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_2013 = cumulative/cumulative.max()

    y = 2014
    ws = ws_good['hourly_%d_%d' % (m,y)]
    #bins = round(ws.max())
    values, base14 = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_2014 = cumulative/cumulative.max()

    y = 2015
    ws = ws_good['hourly_%d_%d' % (m,y)]
    #bins = round(ws.max())
    values, base15 = np.histogram(ws, bins = bins, density=True)
    cumulative = np.cumsum(values)
    cumu_2015= cumulative/cumulative.max()


    #plt.plot(base[:-1], values, 'g-')
    c_09, = plt.plot(base09[:-1], (1.-cumu_2009)*100., 'm-', linewidth = 2)
    c_10, = plt.plot(base10[:-1], (1.-cumu_2010)*100., 'c-', linewidth = 2)
    c_11, = plt.plot(base11[:-1], (1.-cumu_2011)*100., 'y-', linewidth = 2)
    c_12, = plt.plot(base12[:-1], (1.-cumu_2012)*100., 'k-', linewidth = 2)
    c_13, = plt.plot(base13[:-1], (1.-cumu_2013)*100., 'g-', linewidth = 2)
    c_14, = plt.plot(base14[:-1], (1.-cumu_2014)*100., 'r-', linewidth = 2)
    c_15, = plt.plot(base15[:-1], (1.-cumu_2015)*100., 'b-', linewidth = 2)

    plt.title('%s Prob(Hourly Average Wind Speed $>$ x)' % month[m-1])
    plt.xlabel('Wind Speed (knots)')
    plt.ylabel('Cumulative Distribution (%)')
    plt.legend([c_15, c_14, c_13, c_12, c_11, c_10, c_09], ['2015', '2014', '2013', '2012', '2011', '2010', '2009'], loc = 'upper right')

    #plt.xlim([0,ws.max()])
    plt.ylim([0, 105])
    plt.savefig('../meteorology/plots/year_to_year_comparisons/%s/%s_wind_speed_hourly_average_1-cumulative_dist.png' % (month[m-1].lower(), month[m-1].lower()))
    #plt.show()
    plt.close('all')

In [27]:
print(round(-6.4 - -2.2))

-4


In [47]:
#search for this
for y in YEAR:
    #start = int(months['hourly_%d' % y][0])
    #end = int(months['hourly_%d' % y][-1])
    for c in range(6,9):
        #bins = maximum / 0.5
        ws = temperature['hourly_%d_%d' % (c, y)]
        maximum = ws.max()
        minimum = ws.min()
        #weights = np.ones_like(ws)/float(len(ws))
        #bins = round(maximum)*4.
        bins = abs(round(maximum - minimum)*2.)
        #z,x, _ = plt.hist(ws, bins = bins, range = (0,maximum), normed = True) 
        z,x, _ = plt.hist(ws, bins = bins, range = (minimum, maximum), normed = True) 
        plt.title('%s %d Hourly Average Temperature, bin width = %.2f ' % (month[c-1], y, x[1] - x[0]))
        imax = z.tolist().index(z.max())
        #print 'year: ', y, ", month: ", c, ", most frequent speed: ", x[imax]
        #print "-----"
        #plt.ylabel('Wind Speed (m/s)')
        plt.xlabel('Temperature (degrees)')
        plt.ylabel('Probability per Bin')
        #plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
        plt.ylim([0, 0.11])
        plt.xlim([-40., 40.])
        plt.savefig('../meteorology/plots/%d/%s/%s_%d_temperature_hourly_average_normed_hist.png' % (y, month[c-1].lower(), month[c-1].lower(), y))
        #plt.show()
        plt.close('all')

In [11]:
#week by week

weeklyMean = defaultdict(list)
weeklySTD = defaultdict(list)

for y in YEAR:
    
    if y == 2008:
        start = 6 
    else:
        start = 1
        
    for c in range(start, 13):

        #perday['hourly_%d_%d' % (c, y)]
        #wind_speed['hourly_%d_%d' % (c, y)]

        for i in range(4):        
            if i < 3:
                perday['hourly_%d_%d_%d' % (i+1, c, y)] = perday['hourly_%d_%d' % (c, y)][i*24*7:24*7*(i+1)]
                wind_speed['hourly_%d_%d_%d' % (i+1, c, y)] = wind_speed['hourly_%d_%d' % (c, y)][i*24*7:24*7*(i+1)]
                
            else:
                perday['hourly_%d_%d_%d' % (i+1, c, y)] = perday['hourly_%d_%d' % (c, y)][i*24*7:]
                wind_speed['hourly_%d_%d_%d' % (i+1, c, y)] = wind_speed['hourly_%d_%d' % (c, y)][i*24*7:]

            weeklyMean['%d_%d_%d' % (i+1, c, y)].append(wind_speed['hourly_%d_%d_%d' % (i+1, c, y)].mean())
            weeklySTD['%d_%d_%d' % (i+1, c, y)].append(wind_speed['hourly_%d_%d_%d' % (i+1, c, y)].std())

In [249]:
#create arrays with missing elements deleted
#dont forget that you converted to knots
for y in YEAR:
    for m in range(1,13):
        for i in range(1,5):
            w = wind_speed['hourly_%d_%d_%d' % (i,m,y)]

            igood = np.where(w >= 0.)[0]

            ws = np.empty(len(igood))
            ws_good['hourly_%d_%d_%d' % (i,m,y)] = w[igood]*1.94384

In [80]:
#plot weekly for whole year
for y in YEAR[1:]:
    
    weekAvg = np.empty(0)
    
    if y == 2008:
        start = 6 
    else:
        start = 1
    
    for c in range(start, 13):
        for i in range(4): 
            weekAvg = np.hstack([weekAvg, weeklyMean['%d_%d_%d' % (i+1, c, y)]])
    
    numWeeks = np.arange(start, start + len(weekAvg))
    plt.plot(numWeeks, weekAvg, 'b-', linewidth = 2)
    plt.title('%d Weekly Mean' % y)
    plt.xlabel('week')
    plt.ylabel('wind speed (m/s)')
    plt.savefig('../meteorology/plots/%d/%d_weekly_mean.png' % (y, y))
    #plt.show()
    plt.close('all')

In [60]:
#c = 7
#week = 1
for c in range(1,13):
    if c < 7:
        year = YEAR[1:]
    else:
        year = YEAR
        
    for week in range(1,5):
        mean = []
        sig = []
        p = []
        for y in year:
            if y == 2013 and c == 7 and (week == 3 or week == 4):
                mean.append(-99)
                sig.append(-99)
                p.append(-99)
            elif y == 2013 and c == 8 and (week == 1 or week == 2 or week == 3):
                mean.append(-99)
                sig.append(-99)
                p.append(-99)
            else:
                mean.append(weeklyMean['%d_%d_%d' % (week, c, y)][0])
                sig.append(weeklySTD['%d_%d_%d' % (week, c, y)][0])
                p.append(peak['%d_%d_%d' % (week, c, y)])

        mean = ma.masked_equal(mean, -99)
        sig = ma.masked_equal(sig, -99)
        p = ma.masked_equal(p, -99)
        
        plt.plot(year, mean, linewidth = 2)
        plt.xlabel('year')
        plt.ylabel('wind speed (m/s)')
        plt.title('%s week %d mean wind speed for %d-2015' % (month[c-1], week, year[0]))
        plt.savefig('../meteorology/plots/year_to_year_comparisons/%s/week%d/%s_week%d_%d-2015_weekly_mean.png' % (month[c-1].lower(), week, month[c-1].lower(), week, year[0]))
        #plt.show()
        plt.close('all')

        plt.plot(year, sig, linewidth = 2)
        plt.xlabel('year')
        plt.ylabel('wind speed (m/s)')
        plt.title('%s week %d wind speed standard deviation for %d-2015' % (month[c-1], week, year[0]))
        plt.savefig('../meteorology/plots/year_to_year_comparisons/%s/week%d/%s_week%d_%d-2015_weekly_std.png' % (month[c-1].lower(), week, month[c-1].lower(), week, year[0]))
        #plt.show()
        plt.close('all')
        
        plt.plot(year, p, linewidth = 2)
        plt.xlabel('year')
        plt.ylabel('wind speed (m/s)')
        plt.title('%s week %d most frequent wind speed for %d-2015' % (month[c-1], week, year[0]))
        plt.savefig('../meteorology/plots/year_to_year_comparisons/%s/week%d/%s_week%d_%d-2015_most_frequent.png' % (month[c-1].lower(), week, month[c-1].lower(), week, year[0]))
        #plt.show()
        plt.close('all')
        

In [ ]:
#find max, mean, and std wind speed each day (in each month of each year)
#for average and std can use hourly average, for max should use minute by minute

In [38]:

minute = dict()
minuteSP = dict()
minuteH = dict()

In [39]:
#method 1
#time = dict()
for y in YEAR:
        
    if y == 2008:
        start = 6 
    else:
        start = 1
        
    for j in range(start, 13):
        if j < 10:
            M = '0%d' % j
        elif j >= 10:
            M = '%d' % j
        file1 =  '../meteorology/data/summit/%d/minutes/met_sum_insitu_1_obop_minute_%d_%s.txt' % (y, y, M)
        #fileSP = '../meteorology/data/SP/%d/minutes/met_spo_insitu_1_obop_minute_%d_%s.txt' % (y, y, M)
        #fileH =  '../meteorology/data/Hawaii/%d/minutes/met_mlo_insitu_1_obop_minute_%d_%s.txt' % (y, y, M)
        
        results = []
        with open(file1) as inputfile:
            for line in inputfile:
                results.append(line.strip().split())

        direction = []
        speed = []
        temp = []
        d = []
        dSP = []
        dH = []
        h = []
        hSP = []
        hH = []
        m = []
        mSP = []
        mH = []
        #times = []

        for i in range(len(results)):
            direction.append(float(results[i][6]))
            speed.append(float(results[i][7]))
            temp.append(float(results[i][10]))
            d.append(float(results[i][3]))
            h.append(float(results[i][4]))
            m.append(float(results[i][5]))
            


        #wind_dir['minute_%d_%d' % (j,y)] = np.asarray(direction)
        #wind_speed['minute_%d_%d' % (j,y)] = np.asarray(speed)
        temperature['minute_%d_%d' % (j,y)] = np.asarray(temp)

        day['minute_%d_%d' % (j,y)] = np.asarray(d)
        hour['minute_%d_%d' % (j,y)] = np.asarray(h)
        minute['minute_%d_%d' % (j,y)] = np.asarray(m)
        '''
        #time['hourly_%d' % y] = np.arange(1., 13., 12./len(wind_speed['hourly_%d' % y]))
        resultsSP = []
        with open(fileSP) as inputfile:
            for line in inputfile:
                resultsSP.append(line.strip().split())

        resultsH = []
        with open(fileH) as inputfile:
            for line in inputfile:
                resultsH.append(line.strip().split())

        speedSP = []
        speedH = []

        for i in range(len(resultsSP)):
            speedSP.append(float(resultsSP[i][7]))
            dSP.append(float(resultsSP[i][3]))
            hSP.append(float(resultsSP[i][4]))
            mSP.append(float(resultsSP[i][5]))
        for i in range(len(resultsH)):
            speedH.append(float(resultsH[i][7]))
            dH.append(float(resultsH[i][3]))
            hH.append(float(resultsH[i][4]))
            mH.append(float(resultsH[i][5]))

        wind_speedSP['minute_%d_%d' % (j,y)] = np.asarray(speedSP)
        wind_speedH['minute_%d_%d' % (j,y)] = np.asarray(speedH)

        daySP['minute_%d_%d' % (j,y)] = np.asarray(dSP)
        hourSP['minute_%d_%d' % (j,y)] = np.asarray(hSP)
        minuteSP['minute_%d_%d' % (j,y)] = np.asarray(mSP)
        
        dayH['minute_%d_%d' % (j,y)] = np.asarray(dH)
        hourH['minute_%d_%d' % (j,y)] = np.asarray(hH)
        minuteH['minute_%d_%d' % (j,y)] = np.asarray(mH)
        
        '''

In [40]:

for y in YEAR:
    if y == 2008:
        start = 6 
    else:
        start = 1
    for c in range(start,13):
        #wind_speed['minute_%d_%d' % (c, y)] = ma.masked_equal(wind_speed['minute_%d_%d' % (c, y)], -99.9)
        temperature['minute_%d_%d' % (c, y)] = ma.masked_equal(temperature['minute_%d_%d' % (c, y)], -999.9)
        #wind_speedSP['minute_%d_%d' % (c, y)] = ma.masked_equal(wind_speedSP['minute_%d_%d' % (c, y)], -99.9)
        #wind_speedSP['minute_%d_%d' % (c, y)] = ma.masked_equal(wind_speedSP['minute_%d_%d' % (c, y)], -999.9)
        #wind_speedH['minute_%d_%d' % (c, y)] = ma.masked_equal(wind_speedH['minute_%d_%d' % (c, y)], -999.9)
        #wind_dir['minute_%d_%d' % (c, y)] = ma.masked_equal(wind_dir['minute_%d_%d' % (c, y)], -999.)

In [41]:
temp = dict()

In [42]:
for y in YEAR:
    for m in range(1,13):
        #print(y, m)
        #for i in range(len(temperature['minute_%d_%d' % (c, y)])):
        #temperature['minute_%d_%d' % (c, y)][:] = temperature['minute_%d_%d' % (c, y)]* 9. / 5. + 32.
        temp['minute_%d_%d' % (m, y)] = temperature['minute_%d_%d' % (m, y)]* 9. / 5. + 32.
        #temperature['minute_%d_%d' % (c, y)] = t

In [43]:
print(temperature['minute_%d_%d' % (3, 2015)][0])
print(temp['minute_%d_%d' % (3, 2015)][0])

-46.2
-51.16


In [44]:
#find daily max speed for each month
dailyMax = defaultdict(list)
dailyMax_y = defaultdict(list)

for y in YEAR:
    dailyMax_y['%d' % y] = np.empty(0)
    for c in range(1, 13):
        newDay = []
        for i in range(len(wind_speed['minute_%d_%d' % (c, y)])):
            if i > 0 and day['minute_%d_%d' % (c,y)][i] != day['minute_%d_%d' % (c,y)][i-1]:
                newDay.append(i)

        for d in range(len(newDay)):
            if d == 0:
                dailyMax['%d_%d' % (c, y)].append(wind_speed['minute_%d_%d' % (c, y)][0:newDay[d]].max())
            else:
                dailyMax['%d_%d' % (c, y)].append(wind_speed['minute_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].max())

        dailyMax['%d_%d' % (c, y)].append(wind_speed['minute_%d_%d' % (c, y)][newDay[-1]:].max())
        dailyMax_y['%d' % y] = np.hstack([dailyMax_y['%d' % y], dailyMax['%d_%d' % (c, y)]])

KeyError: 'minute_1_2009'

In [45]:
#find daily min speed for each month
dailyMin = defaultdict(list)
#dailyMax_y = defaultdict(list)

for y in YEAR:
    #dailyMax_y['%d' % y] = np.empty(0)
    for c in range(1, 13):
        newDay = []
        for i in range(len(temp['minute_%d_%d' % (c, y)])):
            if i > 0 and day['minute_%d_%d' % (c,y)][i] != day['minute_%d_%d' % (c,y)][i-1]:
                newDay.append(i)

        for d in range(len(newDay)):
            if d == 0:
                dailyMin['%d_%d' % (c, y)].append(temp['minute_%d_%d' % (c, y)][0:newDay[d]].min())
            else:
                dailyMin['%d_%d' % (c, y)].append(temp['minute_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].min())

        dailyMin['%d_%d' % (c, y)].append(temp['minute_%d_%d' % (c, y)][newDay[-1]:].min())
        #dailyMax_y['%d' % y] = np.hstack([dailyMax_y['%d' % y], dailyMax['%d_%d' % (c, y)]])

In [113]:
#find daily max speed for each month
dailyMaxSP = defaultdict(list)
dailyMaxSP_y = defaultdict(list)

for y in YEAR:
    dailyMaxSP_y['%d' % y] = np.empty(0)
    for c in range(1, 13):
        newDay = []
        for i in range(len(wind_speedSP['minute_%d_%d' % (c, y)])):
            if i > 0 and daySP['minute_%d_%d' % (c,y)][i] != daySP['minute_%d_%d' % (c,y)][i-1]:
                newDay.append(i)

        for d in range(len(newDay)):
            if d == 0:
                dailyMaxSP['%d_%d' % (c, y)].append(wind_speedSP['minute_%d_%d' % (c, y)][0:newDay[d]].max())
            else:
                dailyMaxSP['%d_%d' % (c, y)].append(wind_speedSP['minute_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].max())

        dailyMaxSP['%d_%d' % (c, y)].append(wind_speedSP['minute_%d_%d' % (c, y)][newDay[-1]:].max())
        dailyMaxSP_y['%d' % y] = np.hstack([dailyMaxSP_y['%d' % y], dailyMaxSP['%d_%d' % (c, y)]])

In [143]:
print numDays

[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]


In [114]:
#find daily max speed for each month
dailyMaxH = defaultdict(list)
dailyMaxH_y = defaultdict(list)

for y in YEAR:
    dailyMaxH_y['%d' % y] = np.empty(0)
    for c in range(1, 13):
        newDay = []
        for i in range(len(wind_speedH['minute_%d_%d' % (c, y)])):
            if i > 0 and dayH['minute_%d_%d' % (c,y)][i] != dayH['minute_%d_%d' % (c,y)][i-1]:
                newDay.append(i)

        for d in range(len(newDay)):
            if d == 0:
                dailyMaxH['%d_%d' % (c, y)].append(wind_speedH['minute_%d_%d' % (c, y)][0:newDay[d]].max())
            else:
                dailyMaxH['%d_%d' % (c, y)].append(wind_speedH['minute_%d_%d' % (c, y)][newDay[d-1]:newDay[d]].max())

        dailyMaxH['%d_%d' % (c, y)].append(wind_speedH['minute_%d_%d' % (c, y)][newDay[-1]:].max())
        dailyMaxH_y['%d' % y] = np.hstack([dailyMaxH_y['%d' % y], dailyMaxH['%d_%d' % (c, y)]])

In [46]:
#plot max speed for each day in each month
plt.figure(figsize=(10,5))

for c in range(6, 9):
    plt.figure(figsize=(10,5))
    y = 2015
    y2015, = plt.plot(np.arange(len(dailyMin['%d_%d' % (c, y)])), np.asarray(dailyMin['%d_%d' % (c, y)])*scale, 'b-', linewidth = 2)
    y = 2014
    y2014, = plt.plot(np.arange(len(dailyMin['%d_%d' % (c, y)])), np.asarray(dailyMin['%d_%d' % (c, y)])*scale, 'r-', linewidth = 2)
    y = 2013
    y2013, = plt.plot(np.arange(len(dailyMin['%d_%d' % (c, y)])), np.asarray(dailyMin['%d_%d' % (c, y)])*scale, 'g-', linewidth = 2)
    y = 2012
    y2012, = plt.plot(np.arange(len(dailyMin['%d_%d' % (c, y)])), np.asarray(dailyMin['%d_%d' % (c, y)])*scale, 'y-', linewidth = 2)

    plt.legend([y2015, y2014, y2013, y2012], ['2015', '2014', '2013', '2012'], loc = 'upper right')

    plt.title('%s Daily Temperature Minimum' % month[c-1])
    plt.xlabel('Day')
    plt.ylabel('degrees (Fahrenheit)')
    plt.ylim([-40,30])
    #plt.show()
    plt.savefig('../meteorology/plots/year_to_year_comparisons/%s/%s_2012-2015_daily_temperature_min.png' % 
                (month[c-1].lower(), month[c-1].lower()))
    plt.close('all')

In [116]:
#plot daily max speed for each day in whole year
for y in YEAR:
    
    dayMax = np.empty(0)
    
    if y == 2008:
        start = 6 
    else:
        start = 1
    
    for c in range(start, 13):
        dayMax = np.hstack([dayMax, dailyMax['%d_%d' % (c, y)]])
    dailyMax['year_%d' % y] = dayMax

In [118]:
plt.figure(figsize=(15,5))
y = 2015
y2015, = plt.plot(np.arange(len(dailyMax['year_%d' % y])), dailyMax['year_%d' % y]*scale, 'b-', linewidth = 1)
y = 2014
y2014, = plt.plot(np.arange(len(dailyMax['year_%d' % y])), dailyMax['year_%d' % y]*scale, 'r-', linewidth = 1)
y = 2013
y2013, = plt.plot(np.arange(len(dailyMax['year_%d' % y])), dailyMax['year_%d' % y]*scale, 'g-', linewidth = 1)
y = 2012
y2012, = plt.plot(np.arange(len(dailyMax['year_%d' % y])), dailyMax['year_%d' % y]*scale, 'y-', linewidth = 1)

plt.legend([y2015, y2014, y2013, y2012], ['2012', '2011', '2010', '2009'], loc = 'upper right')
#plt.legend([y2013, y2012], ['2010', '2009'], loc = 'upper right')

plt.title('2012-2015 Daily Wind Speed Maximum')
plt.xlabel('Day')
plt.ylabel('Max speed (%s)' % units)
plt.ylim([0,60])
#plt.show()
plt.savefig('../meteorology/plots/year_to_year_comparisons/2012-2015_daily_wind_speed_max_%s.png' % sunits)
plt.close('all')